In [12]:
%matplotlib inline

In [41]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import mpld3
import ezhc as hc

from scipy.optimize import minimize
from sklearn import linear_model
from tools import make_track

sns.set()
#mpld3.enable_notebook()

<IPython.core.display.Javascript object>

Open console to check js libs were loaded in page


In [14]:
prices = pd.read_csv(r"financial_data/prices.csv", index_col=0)
prices.index = pd.DatetimeIndex(prices.index)

mondays = pd.date_range(start=dt.date(2010,1,4), end=dt.date.today(), freq='7D')
returns = prices.reindex(mondays).ffill().pct_change().dropna()

sx5e = returns[["SX5E"]]
bch = returns.drop("SX5E", axis=1)

In [20]:
dates = returns.index.copy() 
n= len(dates)

sample_period = 50
reg_freq = 13

In [21]:
# Contraints linear regression with all coeff positive and sum = 1


a = np.array([1.2, 2.3, 4.2])
b = np.array([1, 5, 6])
c = np.array([5.4, 6.2, 1.9])

m = np.vstack([a,b,c])
y = np.array([5.3, 0.9, 5.6])

def loss(x):
    return np.sum(np.square((np.dot(x, m) - y)))

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0})

x0 = np.zeros(m.shape[0])
res = minimize(loss, x0, method='SLSQP', constraints=cons,
               bounds=[(0, np.inf) for i in range(m.shape[0])], options={'disp': False})

print(res.x)
print(np.dot(res.x, m.T))
print(np.sum(np.square(np.dot(res.x, m) - y)))

[  7.76088097e-01   2.89122408e-14   2.23911903e-01]
[ 1.87173571  2.11955951  4.61630834]
18.817792344


In [22]:
df_weight = pd.DataFrame(columns=bch.columns)

for i in range((n-sample_period)//reg_freq):
    
    start = dates[i*reg_freq]
    end= dates[i*reg_freq+sample_period]
    
    m = bch.loc[start:end].values
    y = sx5e.loc[start:end].values
    
    def loss(x):
        return np.sum(np.square((np.dot(m,x) - y)))

    cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0})
    
    x0 = np.zeros(m.shape[1])
    
    res = minimize(loss, x0, method='SLSQP', constraints=cons,
               bounds=[(0, np.inf) for i in range(m.shape[1])], options={'disp': False})

    df_weight.loc[end] = res.x

df_weight.head()

,IBEX,DAX,FSTEMIB,CAC 40
2010-12-27,1.214306e-16,0.000000e+00,1.000000,1.040834e-16
2011-03-28,0.000000e+00,0.000000e+00,1.000000,0.000000e+00
2011-06-27,0.000000e+00,3.469447e-17,1.000000,0.000000e+00
2011-09-26,0.000000e+00,7.555053e-01,0.155519,8.897593e-02
2011-12-26,2.043721e-16,3.579687e-01,0.231549,4.104823e-01


In [37]:
#prices_for_track = prices.loc[df_weight.index[0]:].drop("SX5E",axis=1)
prices_for_track = prices.loc[df_weight.index[0]:].drop("SX5E",axis=1)
replication = make_track(prices_for_track,df_weight)

df_res = prices.loc[df_weight.index[0]:][["SX5E"]]
df_res["OLS"] = replication

df_res = df_res/df_res.iloc[0]

In [38]:
df_res.head()

,SX5E,OLS
2010-12-27,1.000000,1.000000
2010-12-28,0.999218,1.000197
2010-12-29,1.005208,1.003549
2010-12-30,0.993112,0.991889
2010-12-31,0.993112,0.991889


In [45]:
g = hc.Highstock()

g.chart.height = 550
g.legend.enabled = True
g.legend.layout = 'horizontal'
g.legend.align = 'center'
g.legend.maxHeight = 100
g.tooltip.enabled = True
g.tooltip.valueDecimals = 2
g.exporting.enabled = True

g.chart.zoomType = 'xy'
g.title.text = 'Time series plotted with HighStock'
g.subtitle.text = 'Transparent access to the underlying js lib'

g.plotOptions.series.compare = 'percent'
g.yAxis.labels.formatter = hc.scripts.FORMATTER_PERCENT
g.tooltip.pointFormat = hc.scripts.TOOLTIP_POINT_FORMAT_PERCENT
g.tooltip.positioner = hc.scripts.TOOLTIP_POSITIONER_CENTER_TOP

g.xAxis.gridLineWidth = 1.0
g.xAxis.gridLineDashStyle = 'Dot'
g.yAxis.gridLineWidth = 1.0
g.yAxis.gridLineDashStyle = 'Dot'

g.credits.enabled = True
g.credits.text = 'Source: XXX Flow Strategy & Solutions.'
g.credits.href = 'http://www.example.com'


g.series = hc.build.series(df_res)


g.plot_with_table_1(save=False, save_name='NoTable')

In [40]:
df_weight.applymap(lambda x : np.round(x,3))

,IBEX,DAX,FSTEMIB,CAC 40
2010-12-27,0.000,0.000,1.000,0.000
2011-03-28,0.000,0.000,1.000,0.000
2011-06-27,0.000,0.000,1.000,0.000
2011-09-26,0.000,0.756,0.156,0.089
2011-12-26,0.000,0.358,0.232,0.410
2012-03-26,0.000,0.265,0.117,0.618
2012-06-25,0.000,0.256,0.175,0.569
2012-09-24,0.000,0.000,1.000,0.000
2012-12-24,0.000,0.246,0.754,0.000
2013-03-25,0.000,0.182,0.818,0.000
